In [16]:
import pandas as pd # data processing
import numpy as np # working with arrays
import matplotlib.pyplot as plt # visualization
import seaborn as sb # visualization

from sklearn.model_selection import train_test_split # data split
from sklearn.linear_model import LinearRegression # OLS algorithm

from sklearn.metrics import explained_variance_score as evs # evaluation metric
from sklearn.metrics import r2_score as r2 # evaluation metric
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn import svm
from sklearn.svm import SVC

from feature_engine.encoding import RareLabelEncoder

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from agrupar_categorias import create_pipeline

sb.set_style('whitegrid') # plot style

In [17]:
#LINHA DE SELEÇÃO DO INPUT
df = pd.read_csv('../cade_clinicas_2.csv', low_memory=False)

df.columns


Index(['id', 'id_unique', 'numero_sei', 'ano_documento', 'assinaturas',
       'descricao_tipo_acesso', 'descricao_tipo_documento',
       'descricao_tipo_processo', 'numero_processo', 'id_unidade',
       'sigla_unidade', 'descricao_unidade', 'data_ordem', 'data_documento',
       'setor_economico', 'setor_economico_classe', 'setor_economico_divisao',
       'setor_economico_grupo', 'setor_economico_secao',
       'setor_economico_sub_classe', 'partes_processos_like',
       'partes_processos', 'decisao_sg', 'dados_processo',
       'ementa_dispositivo', 'ementa_verbetacao', 'id_colecao',
       'link_documento', 'mercado_relevante', 'documentos_relacionados',
       'qtd_acesso_processo', 'descricao_tipo_protocolo',
       'descricao_especificacao', 'data_processo', 'colecao', '_version_',
       'palavras', 'relator', 'score', 'nucluster', 'sumario',
       'representante_legal', 'descricao_assunto', 'quorum', 'tags',
       'entidade', 'numero_acordao', 'interessados', 'numero_ata'

In [18]:
# Mudando o tipo para 'category'
colunas = ['descricao_tipo_documento', 'decisao_tribunal', 	'setor_economico', 'setor_economico_sub_classe', 'mercado_relevante', 'assinaturas']
for coluna in colunas:
    df[coluna] = df[coluna].astype('category')

#Formatando colunas de data

df['data_ordem'] = pd.to_datetime(df['data_ordem']).dt.date
df['data_processo'] = pd.to_datetime(df['data_processo'])
df['data_documento'] = pd.to_datetime(df['data_documento'])

# Calcular diferença da data do processo pra data do documento em dias
df['diferenca_dias'] = (df['data_documento'] - df['data_processo']).dt.days

# Remover a hora (deixar apenas a data)
df['data_processo'] = df['data_processo'].dt.date
df['data_documento'] = df['data_documento'].dt.date

Filtro por Voto

In [19]:
tipos_validos = [
    "Voto",
    "Voto Processo Administrativo",
    "Voto Embargos de Declaração"
]

df_votos_filtrados = df.loc[
    (df["descricao_tipo_documento"].isin(tipos_validos)) &
    (df["descricao_tipo_processo"] == "Processo Administrativo")
]

counts = df_votos_filtrados["decisao_tribunal"].value_counts()
total = counts.sum()
print(total)


908


# Limpando colunas:

#### Aqui será feito a seleção das melhores colunas para montar um modelo preditivo, e com isso, as colunas com categorias mais complexas como a do setor_economico serão transformadas para outras colunas numéricas ou com diferentes categorias das que existem agora para um melhor agrupamento e análise desses dados


In [20]:
colunas_manter = [ 'id', 'ano_documento', 'assinaturas', 'descricao_tipo_documento',
                  'decisao_tribunal', 'setor_economico', 'id_unidade', 'data_processo',
                    'data_documento', 'diferenca_dias', 'mercado_relevante','descricao_especificacao', 'conteudo'      
]

df_curto = df_votos_filtrados[colunas_manter]

In [21]:
df_curto['decisao_tribunal'] = (
    df_curto['decisao_tribunal']
    .cat.rename_categories(lambda x: x.replace("['", "").replace("']", "").replace("'", ""))
)

df_curto['assinaturas'] = (
    df_curto['assinaturas']
    .cat.rename_categories(lambda x: x.replace("['", "").replace("']", "").replace("'", ""))
)

df_curto['setor_economico'] = (
    df_curto['setor_economico']
    .cat.rename_categories(lambda x: x.replace("['", "").replace("']", "").replace("'", ""))
)

df_curto['mercado_relevante'] = (
    df_curto['mercado_relevante']
    .cat.rename_categories(lambda x: x.replace("['", "").replace("']", "").replace("'", ""))
)

C:\Users\Danilo\AppData\Local\Temp\ipykernel_19380\793263355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_curto['decisao_tribunal'] = (
C:\Users\Danilo\AppData\Local\Temp\ipykernel_19380\793263355.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_curto['assinaturas'] = (
C:\Users\Danilo\AppData\Local\Temp\ipykernel_19380\793263355.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [22]:
df_curto['decisao_tribunal'] = df_curto['decisao_tribunal'].cat.rename_categories({
    'Condenação': 'condenacao',
    'Arquivamento': 'arquivamento',
    '': 'vazio'
})

df_curto['setor_economico'] = df_curto['setor_economico'].cat.rename_categories({
    '': 'vazio'
})

df_curto['mercado_relevante'] = df_curto['mercado_relevante'].cat.rename_categories({
    '': 'vazio'
})

C:\Users\Danilo\AppData\Local\Temp\ipykernel_19380\735382265.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_curto['decisao_tribunal'] = df_curto['decisao_tribunal'].cat.rename_categories({
C:\Users\Danilo\AppData\Local\Temp\ipykernel_19380\735382265.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_curto['setor_economico'] = df_curto['setor_economico'].cat.rename_categories({
C:\Users\Danilo\AppData\Local\Temp\ipykernel_19380\735382265.py:11: SettingWithCopyWarning: 
A value is trying to be set

Melhorando a informação sobre o setor economico:

Afim de ter dados mais interessantes para o modelo preditivo, vamos transformar os setores economicos para uma nova coluna que se refere a quantas vezes esse setor economico aparece durante a base de dados podendo assim analisar uma correlação entre a quantidade de vezes que um setor economico específico aparece em processos com as condenações nos processos

In [23]:
#Reseta index para modelo rodar semerros
df_curto = df_curto.reset_index(drop=True)

pipeline = create_pipeline(6)
df_curto['setor_economico'] = df_curto['setor_economico'].astype(str)
df_curto['grupo_economico'] = pipeline.fit_transform(df_curto[['setor_economico']])

In [24]:
# Configurar encoder
encoder = RareLabelEncoder(
    tol=0.01,               # Threshold de 10%
    n_categories=5,         # Aplica apenas se variável tiver >=5 categorias
    replace_with='Rare',    # Nome da nova categoria
    variables=['setor_economico'],  # Especificar coluna
    missing_values='ignore' # Trata valores faltantes
)

# Treinar e transformar
encoder.fit(df_curto)
df_curto = encoder.transform(df_curto)

In [25]:
print(df_curto.columns)


Index(['id', 'ano_documento', 'assinaturas', 'descricao_tipo_documento',
       'decisao_tribunal', 'setor_economico', 'id_unidade', 'data_processo',
       'data_documento', 'diferenca_dias', 'mercado_relevante',
       'descricao_especificacao', 'conteudo', 'grupo_economico'],
      dtype='object')


In [26]:
#organizando ordem das colunas:
colunas = [ 'id', 'ano_documento','descricao_tipo_documento','decisao_tribunal', 
            'setor_economico', 'grupo_economico','data_processo','data_documento',
            'diferenca_dias', 'assinaturas', 'id_unidade',
            'mercado_relevante','descricao_especificacao', 'conteudo']

df_final = df_curto[colunas]

In [27]:
pd.set_option('display.max_colwidth', 100)
display(df_final.head(5))
print(df_final.shape)

,id,ano_documento,descricao_tipo_documento,decisao_tribunal,setor_economico,grupo_economico,data_processo,data_documento,diferenca_dias,assinaturas,id_unidade,mercado_relevante,descricao_especificacao,conteudo
0,abbdc450-7ff1-4990-9bcc-8ae4386592f0,2024,Voto Processo Administrativo,vazio,Fabricação de Peças e Acessórios para o Sistema Motor de Veículos Automotores,0,2019-09-13,2024-05-23,1714,Outras,110000967.0,"pistões de motor, bronzinas, camisas, pinos, bielas, porta anéis, anéis e juntas de vedação, e a...","- Desmembrado Bernd Brünig, Faustino Luigi Minchella e José Angel Viani Barroyeta",SEI/CADE - 1390607 - Voto Processo Administrativo\n\n\nProcesso Administrativo nº 08700.004558/2...
1,b41e8f1f-449e-4d28-a574-18f365d32b6f,2024,Voto Processo Administrativo,condenacao,Rare,0,2015-05-21,2024-05-09,3276,Gustavo Augusto Freitas de Lima,110000969.0,Mercado imobiliário,Influencia de conduta uniforme - COFECI,SEI/CADE - 1384594 - Voto Processo Administrativo\n\n\nPedido de Reapreciação no Processo Admini...
2,adec625d-4f5e-4514-b938-a295fdc1e8a4,2024,Voto,arquivamento,Rare,0,2022-08-08,2024-05-09,640,Outras,110001021.0,Postos de gasolina,Ministério Público do Estado de Minas Gerais,SEI/CADE - 1384685 - Voto\n\n\nProcesso nº 08700.005915/2022-40\r\n\r\nRepresentante: Cade ex of...
3,c8499708-5778-45d9-8914-9ab97b77c1fd,2024,Voto Processo Administrativo,condenacao,Construção de Edifícios# Construção de Rodovias e Ferrovias,0,2016-11-22,2024-04-23,2709,Alexandre Cordeiro Macedo,110000955.0,obras de engenharia civil# Construção de infraestrutura de rodovias e ferrovias,PAC Favelas - AL 09/2016 - Favelas,SEI/CADE - 1377990 - Voto Processo Administrativo\n\n\nProcesso Administrativo nº 08700.007776/2...
4,29758e3d-7047-4c5a-8ac6-b6815c67ec3d,2024,Voto Processo Administrativo,condenacao,Construção de Edifícios# Construção de Rodovias e Ferrovias,0,2016-11-22,2024-04-23,2709,Gustavo Augusto Freitas de Lima,110000969.0,obras de engenharia civil# Construção de infraestrutura de rodovias e ferrovias,PAC Favelas - AL 09/2016 - Favelas,SEI/CADE - 1377023 - Voto Processo Administrativo\n\n\nProcesso Administrativo nº 08700.007776/2...


(908, 14)


# Modelo preditivo:

#### Aqui será montado um modelo preditivo para fazer inferencias a partir de características do caso e prever a condenação ou não desse processo